In [1]:
import torch
from models import PCN
import sys
sys.path.append('.')

import os
import random
from loguru import logger

import torch
import torch.utils.data as data
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [30]:
def random_sample(pc, n):
    idx = np.random.permutation(pc.shape[0])
    if idx.shape[0] < n:
        idx = np.concatenate([idx, np.random.randint(pc.shape[0], size=n-pc.shape[0])])
    return pc[idx[:n]]

def read_point_cloud(path):
    pcd = o3d.io.read_point_cloud(path)
    return pcd, np.array(pcd.points, np.float32)


def perturbation(pc, ratio=0.01):
    max_points = len(pc.points)
    return random_radius_mask(pc, random.randint(0, max_points-1), 50)
#     if random.random() < 0.5:
#         max_points = len(pc.points)
#         k = random.randint(int(max_points * ratio/4), int(max_points * ratio))

#         return random_radius_mask(pc, random.randint(0, max_points-1), k)
#     else:

#         return random_rect_mask(pc, random.uniform(0.4, 0.6), random.uniform(0.4, 0.6))


def random_radius_mask(pcd, point_index, k):
    points = np.asarray(pcd.points)

    # Build KD tree for KNN search
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Find the indices of the K-nearest neighbors for the specified point index
    k_neighbors_indices = pcd_tree.search_knn_vector_3d(points[point_index], k)[1]

    # Create a mask to filter out points based on KNN indices
    mask = np.isin(np.arange(len(points)), k_neighbors_indices)

    # Remove the masked points from the point cloud
    pcd = pcd.select_by_index(np.where(~mask)[0])
    return pcd

def random_rect_mask(pcd, scale_factor_x, scale_factor_y):
    points = np.asarray(pcd.points)

    # Generate random rectangle coordinates
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])

    x_range = x_max - x_min
    y_range = y_max - y_min

    x1 = random.uniform(x_min, x_max - x_range * scale_factor_x)
    y1 = random.uniform(y_min, y_max - y_range * scale_factor_y)
    x2 = random.uniform(x1 + x_range * scale_factor_x, x_max)
    y2 = random.uniform(y1 + y_range * scale_factor_y, y_max)


    # Create a mask for points inside the rectangle
    mask = (points[:, 0] >= x1) & (points[:, 0] <= x2) & (points[:, 1] >= y1) & (points[:, 1] <= y2)

    # Apply the mask to the point cloud
    masked_points = points[mask]

    # Create a new point cloud with masked points
    masked_cloud = o3d.geometry.PointCloud()
    masked_cloud.points = o3d.utility.Vector3dVector(masked_points) 

    return masked_cloud


def generate_data(complete_path):
    pcd, pcd_points = read_point_cloud(complete_path)
    o3d.visualization.draw_plotly([pcd])
    
    complete_pc = random_sample(pcd_points, 3000)

    partial_pcd = perturbation(pcd)
    o3d.visualization.draw_plotly([partial_pcd])
    
    partial_pc = random_sample(np.array(partial_pcd.points, np.float32), 1000)

    return torch.from_numpy(partial_pc), torch.from_numpy(complete_pc)


In [31]:
p, c =generate_data("/data/puwong/pcn/data/data/SimulationDataset/robot-single/1/NewLabelSim0.pcd")

In [6]:
ckpt_path = "/data/puwong/pcn/PCN/log/simulation-dataset/all/checkpoints/best_l1_cd.pth"

model = PCN(16384, 1024, 4).to('cuda')
model.load_state_dict(torch.load(ckpt_path))
model.eval()

PCN(
  (first_conv): Sequential(
    (0): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  )
  (second_conv): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
  )
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=1024, out_features=3072, bias=True)
  )
  (final_conv): Sequential(
    (0): Conv1d(1029, 512, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inpl

In [17]:
def l2_cd(pcs1, pcs2):
    dist1, dist2 = CD(pcs1, pcs2)
    dist1 = torch.mean(dist1, dim=1)
    dist2 = torch.mean(dist2, dim=1)
    return torch.sum(dist1 + dist2)


def l1_cd(pcs1, pcs2):
    dist1, dist2 = CD(pcs1, pcs2)
    dist1 = torch.mean(torch.sqrt(dist1), 1)
    dist2 = torch.mean(torch.sqrt(dist2), 1)
    return torch.sum(dist1 + dist2) / 2


def f_score(pred, gt, th=0.01):
    """
    References: https://github.com/lmb-freiburg/what3d/blob/master/util.py

    Args:
        pred (np.ndarray): (N1, 3)
        gt   (np.ndarray): (N2, 3)
        th   (float): a distance threshhold
    """
    pred = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pred))
    gt = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(gt))

    dist1 = pred.compute_point_cloud_distance(gt)
    dist2 = gt.compute_point_cloud_distance(pred)

    recall = float(sum(d < th for d in dist2)) / float(len(dist2))
    precision = float(sum(d < th for d in dist1)) / float(len(dist1))
    return 2 * recall * precision / (recall + precision) if recall + precision else 0

In [ ]:
eva_p, eva_c = p, c

In [32]:
import torch
import open3d as o3d

from extensions.chamfer_distance.chamfer_distance import ChamferDistance
from extensions.earth_movers_distance.emd import EarthMoverDistance


CD = ChamferDistance()
EMD = EarthMoverDistance()

total_l1_cd, total_l2_cd, total_f_score = 0.0, 0.0, 0.0


eva_p = p.to('cuda').unsqueeze(0)
eva_c = c.to('cuda').unsqueeze(0)
_, c_ = model(eva_p)

total_l1_cd += l1_cd(c_, eva_c).item()
total_l2_cd += l2_cd(c_, eva_c).item()

for i in range(len(eva_c)):
    input_pc = eva_p[i].detach().cpu().numpy()
    output_pc = c_[i].detach().cpu().numpy()
    gt_pc = eva_c[i].detach().cpu().numpy()
    total_f_score += f_score(output_pc, gt_pc)

    
total_l1_cd, total_l2_cd, total_f_score

(25.200809478759766, 1286.421142578125, 0.0)

In [33]:
c_.squeeze(0).shape

torch.Size([16384, 3])

In [34]:
# Create a new point cloud with masked points
masked_cloud = o3d.geometry.PointCloud()
masked_cloud.points = o3d.utility.Vector3dVector(np.asarray(c_.squeeze(0).detach().cpu().numpy())) 

# Save the masked point cloud as PCD
o3d.visualization.draw_plotly([masked_cloud])